In [1]:
# Loading up the libraries

import numpy as np
import os
import pandas as pd
import datetime
import xarray as xr
import tensorflow as tf
import glob

import matplotlib.pyplot as plt
plt.style.use('ggplot')
%matplotlib inline

In [2]:
os.chdir("C://Users//Greg//code//space physics//experiment//data")

In [3]:
ss_file = "substorms_2000_2018.csv"
ss = pd.read_csv(ss_file)
ss.index = pd.to_datetime(ss.Date_UTC)
ss = ss.drop(columns=['Date_UTC'])

mag_file = "mag_data_2000.nc"
mag_data = xr.open_dataset(mag_file)

f = np.load("statistics.npz")
mean = f['mean']
std = f['std']

In [7]:
mag_data = mag_data.sel(dim_1=['MLT','MLAT','N','E','Z'])
time_idx = mag_data.Date_UTC[0].values
data_dt = np.timedelta64(20, 'm')
predict_dt = np.timedelta64(200, 'm')
chunk = mag_data.sel(Date_UTC=slice(time_idx, time_idx+data_dt))
cond = np.isnan(chunk).any(dim=['Date_UTC', 'dim_1'])
drop = [st for st in cond if cond[st]]
chunk = chunk.drop(drop)
mag_chunk = chunk.sel(dim_1=['N','E','Z']).to_array().values
print("mag chunk shape: ", mag_chunk.shape)

mlt = chunk.sel(dim_1='MLT').to_array()
mlat = chunk.sel(dim_1='MLAT').to_array()[:,0]
sinmlt = np.sin(mlt).mean(dim='Date_UTC')
cosmlt = np.cos(mlt).mean(dim='Date_UTC')
st_loc = np.stack((sinmlt, cosmlt, mlat/90), axis=1)
print("station location shape: ", st_loc.shape)

target_chunk = ss[time_idx+data_dt:time_idx+data_dt+predict_dt]
yesno = len(target_chunk) > 0
first = target_chunk.iloc[0]
time = (first.name - (time_idx+data_dt)).total_seconds() / (200 * 60)
location = (np.sin(first['MLT'] * 2 * np.pi / 24), np.cos(first['MLT'] * 2 * np.pi / 24), first['MLAT']/90)

dataset = mag_data.copy()

da = dataset.to_array().values
sinmlt = np.sin(da[:,:,0])
cosmlt = np.cos(da[:,:,0])
mlat = da[:,:,1] / 90
nez = (da[:,:,2:] - mean) / std

data = np.concatenate((sinmlt[:,:,None], cosmlt[:,:,None], mlat[:,:,None], nez), axis=2)

xr.Dataset({st: (['Date_UTC', 'vals'], data[i]) for i,st in enumerate(dataset)}, 
          coords={'Date_UTC': dataset.Date_UTC, 'vals': ['SINMLT','COSMLT','MLAT','N','E','Z']})


mag chunk shape:  (159, 21, 3)
station location shape:  (159, 3)


<xarray.Dataset>
Dimensions:   (Date_UTC: 507007, vals: 6)
Coordinates:
  * Date_UTC  (Date_UTC) datetime64[ns] 2000-01-01T00:13:00 ... 2000-12-31T23:59:00
  * vals      (vals) <U6 'SINMLT' 'COSMLT' 'MLAT' 'N' 'E' 'Z'
Data variables:
    ABG       (Date_UTC, vals) float64 -0.9996 0.02761 0.1339 ... -0.018 -0.265
    ABK       (Date_UTC, vals) float64 0.9613 -0.2756 ... -0.001211 0.6162
    ALE       (Date_UTC, vals) float64 0.9857 -0.1684 0.9689 ... 0.005414 0.7297
    AMK       (Date_UTC, vals) float64 nan nan nan ... -0.7975 0.01734 0.6521
    AMS       (Date_UTC, vals) float64 -0.9162 -0.4008 -0.5457 ... 0.0222 -1.987
    AND       (Date_UTC, vals) float64 0.984 -0.1782 0.7371 ... 0.004531 0.6199
    API       (Date_UTC, vals) float64 -0.006371 1.0 -0.1727 ... 0.07697 -1.263
    APL       (Date_UTC, vals) float64 0.1202 0.9928 0.5553 ... nan nan nan
    ASC       (Date_UTC, vals) float64 -0.7166 -0.6975 ... 0.0005554 -1.236
    ASP       (Date_UTC, vals) float64 0.5268 -0.85 -0.3772

In [3]:
def make_generator(mag_files, ss_file, stats_file, data_interval, prediction_interval):
    """My data generator feeding function
    
    Inputs:
        mag_files: list of magnetometer data file names
        ss_file: string file containing substorm data
        stats_file: string file containing mean and std
        data_interval: number of minutes for data interval
        prediction_interval: number of minutes for prediction interval
        
    Outputs:
        gen: the generator function
    
    What this needs to do:
        - load the i-th data file
        - strip away the unused variables
        - break the data into batches
            - time series for each station
            - sin(MLT), cos(MLT) and MLAT for each station
            - planet scale parameters
            - labels:
                - yes/no
                - time [0,1] for interval                           ------| for first substorm
                - location [sin(MLT), cos(MLT), MLAT in [-1,1]]     ------|
        - for the station time series':
            - remove stations with too many nans
            - outliers and remaining NaNs
            - remove mean, scale by STD
            - shuffle up stations? other augmentations?
        - yield (station x time x component), MLT, MLAT, planet params, targets
        - when current file is finished, load up next file
    """
    
    stats = np.load(stats_file)
    mean = stats['mean']
    std = stats['std']
    
    ss = pd.read_csv(ss_file)
    ss.index = pd.to_datetime(ss.Date_UTC)
    ss = ss.drop(columns=['Date_UTC'])
    
    data_dt = np.timedelta64(data_interval, 'm')
    predict_dt = np.timedelta64(prediction_interval, 'm')        
    
    def gen():
        
        for file in mag_files:
            print(file)
            
            dataset = xr.open_dataset(file).sel(dim_1=['MLT','MLAT','N','E','Z'])
            
            da = dataset.to_array().values
            sinmlt = np.sin(da[:,:,0])
            cosmlt = np.cos(da[:,:,0])
            mlat = da[:,:,1] / 90
            nez = (da[:,:,2:] - mean) / std

            data = np.concatenate((sinmlt[:,:,None], cosmlt[:,:,None], mlat[:,:,None], nez), axis=2)

            mag_data = xr.Dataset({st: (['Date_UTC', 'vals'], data[i]) for i,st in enumerate(dataset)},
                                  coords={'Date_UTC': dataset.Date_UTC, 'vals': ['SINMLT','COSMLT','MLAT','N','E','Z']})
            
            del dataset
            
            time_idx = mag_data.Date_UTC[0].values
            
            while True:
                
                if time_idx + data_dt > mag_data.Date_UTC[-1].values:
                    break
                
                # select time slice
                chunk = mag_data.sel(Date_UTC=slice(time_idx, time_idx + data_dt))     
                
                # skip if there is missing data
                if chunk.Date_UTC.shape[0] != data_interval + 1:
                    time_idx += data_dt
                    continue
                    
                # filter out stations with NaNs
                cond = np.isnan(chunk).any(dim=['Date_UTC', 'vals'])
                drop = [st for st in cond if cond[st]]
                chunk = chunk.drop(drop)
                
                # mag data
                mag_chunk = chunk.sel(vals=['N','E','Z']).to_array()
                
                # MLT / MLAT data
                mlat = chunk.sel(vals='MLAT').to_array()[:,0]
                sinmlt = chunk.sel(vals='SINMLT').to_array()[:,data_interval//2]
                cosmlt = chunk.sel(vals='COSMLT').to_array()[:,data_interval//2]
                st_loc = np.stack((sinmlt, cosmlt, mlat), axis=1)
                
                # planet scale parameters
                
                # increment time index
                time_idx += data_dt
                # create target
                target_chunk = ss[time_idx:time_idx+predict_dt]
                ss_occurred = len(target_chunk) > 0
                time = 0
                location = (0, 0, 0)
                if ss_occurred:
                    first = target_chunk.iloc[0]
                    time = (first.name - (time_idx)).total_seconds() / (200 * 60)
                    location = (np.sin(first['MLT'] * 2 * np.pi / 24), 
                                np.cos(first['MLT'] * 2 * np.pi / 24), 
                                first['MLAT']/90)
                    
                yield mag_chunk, st_loc, ss_occurred, time, location
                
            del mag_data
                
    return gen

In [7]:
mag_files = glob.glob("mag_data*.nc")
ss_file = ss_file = "substorms_2000_2018.csv"
stats_file = "statistics.npz"
data_interval = 120
prediction_interval = 250

gen = make_generator(mag_files, ss_file, stats_file, data_interval, prediction_interval)

dataset = tf.data.Dataset.from_generator(gen,
                                         output_types = (tf.float32, tf.float32, tf.bool, tf.float32, tf.float32),
                                         output_shapes = (tf.TensorShape([None, data_interval+1, 3]),
                                                          tf.TensorShape([None, 3]),
                                                          tf.TensorShape([]),
                                                          tf.TensorShape([]),
                                                          tf.TensorShape([3]))
                                        )

dataset = dataset.repeat()

iter = dataset.make_initializable_iterator()
el = iter.get_next()

import time
t = time.time()
EPOCHS = 10
with tf.Session() as sess:
    sess.run(iter.initializer)
    for i in range(EPOCHS):
        mag, st_loc, occ, ss_t, ss_loc = sess.run(el)

total_time = time.time() - t
print("Total Time: ", total_time)
print("Time per epoch: ", total_time/EPOCHS)
print(mag.shape)

mag_data_2000.nc
Total Time:  11.662115573883057
Time per epoch:  1.1662115573883056
(165, 121, 3)
